Stocks
Sep-November 2023
Personal Project 

Things I learned doing this project:
    
    Python Data import (CSV)
    
    Jupyter interface/use
    
    Pandas DataFrame use

    Datetime use
    
This code imports a CSV of stock brokerage history data. The script filters the data to only buys and sells. It then calculates the profit or loss than would occur if you sold a specific stock, including whether the sell was a short or long term sale.

In [75]:
import pandas as pd
from datetime import datetime, timedelta

brokerageRawData = pd.read_csv('Sample Data.csv') # Data is random and not my own fincancial information or stock buys/sells

brokerageRawData.head()
# brokerageRawData.describe()

,Activity Date,Process Date,Settle Date,Instrument,Description,Trans Code,Quantity,Price,Amount,Suppressed
0,9/22/2023,9/22/2023,9/26/2023,SPYG,SPDR Portfolio S&P 500 Growth ETF,Buy,7.0,$6,$54,N
1,9/21/2023,9/21/2023,9/21/2023,SPYG,Cash Div: R/D 2023-09-19 P/D 2023-09-21 - 340....,CDIV,-1.0,$10,$10,N
2,9/19/2023,9/19/2023,9/21/2023,SPYG,SPDR Portfolio S&P 500 Growth ETF,Buy,15.0,$7,($42),N
3,9/18/2023,9/18/2023,9/20/2023,NEE,NextEra Energy,Buy,12.0,$8,$24,N
4,9/15/2023,9/15/2023,9/15/2023,NEE,Cash Div: R/D 2023-08-30 P/D 2023-09-15 - 31.1...,CDIV,1.0,$3,($30),N


In [77]:
#Filter out other transactions such as dividends, options, interest, etc.
allStockTransactions = brokerageRawData[ (brokerageRawData['Trans Code'] == 'Buy') | (brokerageRawData['Trans Code'] == 'Sell')] 

# Get a list of all the unique stocks that have been bought
uniqueStockList = pd.Series(allStockTransactions.loc[:,'Instrument']).drop_duplicates().tolist()
print('Unique stocks are: \n' + str(uniqueStockList))

stockName = input('Enter the ticker symbol of the stock you wish to analyze: ')
# stockName = 'TSLA' #Hardcode the previous line if you would rather
StockData = allStockTransactions.loc[allStockTransactions.Instrument == stockName].copy()
    #Create a .copy() so that a SettingWithCopy warning is avoided
    
# StockData.head(10)

# StockData['Price'] = StockData['Price'].astype(str)

# Price/Quantity column is a string, such as $12.34 or 1.5. Need to drop "$" and "," and make numeric
StockData['Quantity'] = StockData['Quantity'].astype(str)
StockData['Price']  = StockData['Price'].str.replace('$','',regex=True)
StockData['Price'] = StockData['Price'].str.replace(',','',regex=True)
StockData['Price'] = StockData['Price'].str.replace('(','',regex=True)
StockData['Price'] = StockData['Price'].str.replace(')','',regex=True)
StockData['Price'] = pd.to_numeric(StockData['Price'])


StockData['Quantity'] = StockData['Quantity'].astype(str)
StockData['Quantity'] = StockData['Quantity'].str.replace(',','',regex=True)
StockData['Quantity'] = pd.to_numeric(StockData['Quantity'])

## Handling Stock Splits
# Some stocks have splits, which would confuse our data. This code can handle splits, but it needs the split date and ratio.
   # For example, Tesla had a stock split on 8/25/22, 3 for 1
   # This code adjusts quantity and price for data before this date so it can be compared to now
   # If transaction was before 8/25/22, quantity = 3x and price = /3
   # Improvements could be made here to handle multiple splits, asking user to input the data/retrieving it from the internet.
stockSplit = 0 #Change this to a 1 and input stock split date and ratio
if stockSplit == 1:
    splitValue1 = 5 # 1 to 5 split
    splitDate1 = '2020-08-31'
    StockData.loc[pd.to_datetime(StockData['Activity Date']) <= splitDate1, 'Quantity'] *= splitValue1
    StockData.loc[pd.to_datetime(StockData['Activity Date']) <= splitDate1, 'Price'] /= splitValue1

    splitValue2 = 3
    splitDate2 = '2022-08-25'
    StockData.loc[pd.to_datetime(StockData['Activity Date']) <= splitDate2, 'Quantity'] *= splitValue2
    StockData.loc[pd.to_datetime(StockData['Activity Date']) <= splitDate2, 'Price'] /= splitValue2

# Fix faulty data
# The company that generated the CSV must have changed their data management at some point. For older transactions, the quantity is not listed as a negative for a sell, whereas it is listed as a negative for a sell with more recent transactions. Fix this by making sure that all transactions with "sell" in the Trans Code have a negative for the corresponding Quantity
StockData.loc[(StockData['Trans Code'] == 'Sell') & (StockData['Quantity'] > 0), 'Quantity'] *= -1


# Now that we have a total and the data corrected, we want to generate a list that would be helpful if you're interested in selling a current stock you hold. 
# Ask input for current stock price (maybe look it up on the internet?). Then, for the stocks that you hold, calculate when they were bought (long term vs short term gain) and the profit for a current sell. 

#Ask for input or hard code current price
# CurrentPrice = float(input('Enter the price for ' + stockName))
CurrentPrice = 213.00 
CurrentPosition = StockData['Quantity'].sum()
i = 1
PositionSummary = []

# Creates a position summary with the buy date, price, and quantity for stocks currently held
for index, row in StockData.iterrows():
    if 'Buy' in row['Trans Code'] and CurrentPosition >= 0:
        PositionSummary.append((row['Activity Date'], row['Quantity'], row['Price']))
        i=i+1
        CurrentPosition = CurrentPosition - row['Quantity'] # Iterates backwards through buy history to the oldest buy that is still held. This code can handle stocks that have been bought/sold multiple times

# Determine if your current position is a long-term position (> 1 year old) or short-term        
OldestBuy = PositionSummary[-1][0] # If an error occurs here, it is probably because PositionSummary is empty (CurrentPosition < 0) for the preceeding for loop. This could never happen with real data, but my made up data could have a negative position possibly..
OldestBuyDate = datetime.strptime(OldestBuy, '%m/%d/%Y')
today = datetime.today()
one_year_ago = today - timedelta(days=365)

if OldestBuyDate < one_year_ago:
    longTermSale = 'Long Term'
else:
    longTermSale = 'Short Term'

currentPositionValue = 0
for TSLA_BuyHistory in PositionSummary:    
    currentPositionValue += TSLA_BuyHistory[1] * TSLA_BuyHistory[2]

Net = CurrentPrice * StockData['Quantity'].sum() - currentPositionValue

print('Net ' + longTermSale + ' profit(loss) if all position sold now: ' + str(Net))

Unique stocks are: 
['SPYG', 'NEE', 'HD', 'MSFT', 'ABNB', 'LOW', 'NOBL', 'TSLA', 'BRLT', 'COIN', 'ESPO', 'BYND', 'BABA', 'ANSS', 'DIS', 'BA', 'ARKG', 'ARKK', 'CCL', 'RCL', 'VXUS', 'TGT', 'QQQ', 'AMZN', 'SQ', 'GOOG', 'NIO', 'XONE', 'MMM', 'TROX', 'NTDOY', 'GS', 'META', 'MODG', 'DHI', 'AAPL']


Enter the ticker symbol of the stock you wish to analyze:  SPYG


Net Long Term profit(loss) if all position sold now: 182079.0
